<a href="https://colab.research.google.com/github/wellia/ComputerVision/blob/main/Pest_Detection_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp "/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset

  3% 323M/11.1G [00:02<01:21, 142MB/s]
User cancelled operation


In [ ]:
import zipfile
with zipfile.ZipFile('/content/arthropod-taxonomy-orders-object-detection-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
#Import statement
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import PIL 
from PIL import Image

import json
import pickle

In [ ]:
#Set the directory path
directory = "/content"
os.chdir(directory)

In [ ]:
def json_to_csv():
    path_to_json = '/content/ArTaxOr/Araneae/annotations'
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    path_to_jpeg = '/content/ArTaxOr/Araneae'
    jpeg_files = [pos_jpeg for pos_jpeg in os.listdir(path_to_jpeg) if pos_jpeg.endswith('.jpg')]
    fjpeg=(list(reversed(jpeg_files)))
    n=0
    csv_list = []
    labels=[]
    for j in json_files:
        data_file=open('/content/ArTaxOr/Araneae/annotations/{}'.format(j))
        data = json.load(data_file)
        asset = data['asset']
        size = asset['size']
        width,height=size['width'],size['height']
        for region in data["regions"]:
            box = region['boundingBox']
            if region['tags']!='None':
                name=region['tags'][0]
                labels.append(name)
                xmin=round(box['left'])
                ymin=round(box['top'])
                bwidth = round(box['width'])
                bheight = round(box['height'])
                value = (fjpeg[n], width, height, name, xmin, ymin, bwidth, bheight)
                csv_list.append(value)
        n=n+1
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'bwidth', 'bheight']
    csv_df = pd.DataFrame(csv_list, columns=column_name)
    print(csv_df)
    # labels_train=list(set(labels))
    # with open("train_labels.txt", "wb") as fp:   #Pickling
    #     pickle.dump(labels_train, fp)
    return csv_df

def main():
    for directory in ['Araneae']:
        csv_df = json_to_csv()
        csv_df.to_csv('{}_labels.csv'.format(directory), index=None)
        print('Successfully converted json to csv.')
    #Set the directory path
    directory = "/drive/MyDrive/Project_Code"
    os.chdir(directory)
    csv_df.to_csv('Araneae_labels.csv', index=None)
main()

              filename  width  height        class  xmin  ymin  bwidth  bheight
0     c69d0b368c18.jpg   1287    2048      Araneae    85   330    1083     1654
1     d0439f5ece00.jpg   2048    1536      Araneae   624   277     884      830
2     0945aacb922d.jpg   1380    2048  Lepidoptera   424   457     771     1306
3     0945aacb922d.jpg   1380    2048      Araneae   318   249     506     1108
4     6be87c2c7729.jpg   1589    2048      Araneae   209   592    1030      738
...                ...    ...     ...          ...   ...   ...     ...      ...
2667  72892e8a5004.jpg   1536    2048      Araneae   172   208    1364     1150
2668  55887a3970bc.jpg   1535    2048      Araneae    57   465    1441     1189
2669  4a3e02dcb315.jpg   2048    1369      Araneae   503   351     927      918
2670  cd20c5fac501.jpg   2048    1536      Araneae   430   402    1011      893
2671  aa5686c7443f.jpg   2048    1536      Araneae   656   374    1035      998

[2672 rows x 8 columns]
Successfully co

In [ ]:
#Read the CSV file to Dataframe
df_imagedetails = pd.read_csv('/content/Araneae_labels.csv') 
df_imagedetails.head() 

,filename,width,height,class,xmin,ymin,bwidth,bheight
0,c69d0b368c18.jpg,1287,2048,Araneae,85,330,1083,1654
1,d0439f5ece00.jpg,2048,1536,Araneae,624,277,884,830
2,0945aacb922d.jpg,1380,2048,Lepidoptera,424,457,771,1306
3,0945aacb922d.jpg,1380,2048,Araneae,318,249,506,1108
4,6be87c2c7729.jpg,1589,2048,Araneae,209,592,1030,738


In [ ]:
#Create new folder
! mkdir /drive/MyDrive/Project_Code/modified_images

In [ ]:
def resize(img, resolution=480, padding=6):
    w , h ,c = img.shape
    from skimage.transform import resize
    return resize(img, (int(resolution*w/h), int(resolution)), preserve_range=True).astype(np.uint8)

def reshape(image, box):
  xmin, ymin, bhight , bwidth = box
  #original shape
  Oh,Ow,c = image.shape

  new_iamge  = resize(image, 640)

  Nh,Nw,c  = new_iamge.shape

  n_minR  = xmin/Ow
  n_yminR = ymin/Oh

  xmin = round(n_minR*Nw)
  ymin = round(n_yminR*Nh)

  xmax = round(xmin + bwidth)
  ymax =  round(ymin + bhight)

  wR = xmax/Ow
  hR = ymax/Oh

  #print("new xmin", xmin)
  #print("new ymin", ymin)
  #print("new width", wR*Nw +xmin)
  #print("new height", hR*Nh +ymin)
  return (xmin,ymin),(round(wR*Nw +xmin),round(hR*Nh +ymin)) , (new_iamge)

In [ ]:
#Set the folder path
folder_path = "/content/ArTaxOr/Araneae/"
directory = "/drive/MyDrive/Project_Code/modified_images"
os.chdir(directory)
  
#color = (255, 0, 0) 
#thickness = 10  
for i in range(len(df_imagedetails)):
  image = cv2.imread(folder_path + df_imagedetails.loc[i, "filename"])
  min , max, new_image = reshape(image,[df_imagedetails.loc[i, "xmin"], df_imagedetails.loc[i, "ymin"],
                                  df_imagedetails.loc[i, "bheight"], df_imagedetails.loc[i, "bwidth"]])
  #new = cv2.rectangle(image, a, b, color, thickness)
  #plt.imshow(new)
  cv2.imwrite(df_imagedetails.loc[i, "filename"], new_image)
  df_imagedetails.loc[i, "xmin"] = min[0]
  df_imagedetails.loc[i, "ymin"] = min[1]
  df_imagedetails.loc[i, "bwidth"] = max[0]
  df_imagedetails.loc[i, "bheight"] = max[1]

#Rename dataframe column
df_imagedetails = df_imagedetails.rename(columns={"bwidth": "xmax", "bheight": "ymax"})
df_imagedetails.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,c69d0b368c18.jpg,1287,2048,Araneae,51,197,731,1304
1,d0439f5ece00.jpg,2048,1536,Araneae,241,107,676,468
2,0945aacb922d.jpg,1380,2048,Lepidoptera,220,237,734,1036
3,0945aacb922d.jpg,1380,2048,Araneae,165,129,513,770
4,6be87c2c7729.jpg,1589,2048,Araneae,160,454,1073,1368


In [ ]:
directory = "/content"
os.chdir(directory)

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
pip install tensorflow-object-detection-api

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
! mkdir /content/tfrecords

mkdir: cannot create directory ‘/content/tfrecords’: File exists


In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
cd /drive/MyDrive/Project_Code/models/research

/drive/MyDrive/Project_Code/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

fatal: destination path 'cocoapi' already exists and is not an empty directory.


In [ ]:
cd cocoapi/PythonAPI

/drive/My Drive/Project_Code/models/research/cocoapi/PythonAPI


In [ ]:
!make

python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-ind

In [ ]:
pwd

'/drive/My Drive/Project_Code/models/research/cocoapi/PythonAPI'

In [ ]:
cp -r pycocotools /drive/MyDrive/Project_Code/models/research

In [ ]:
cd /drive/MyDrive/Project_Code/models/research

/drive/MyDrive/Project_Code/models/research


In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

Processing /drive/My Drive/Project_Code/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1660787 sha256=980989e73b0947ffe5faf9cc599e1ecf4790dd011e16d5fc84baa30f4aa4a22c
  Stored in directory: /tmp/pip-ephem-wheel-cache-iirdjkd7/wheels/5e/3d/90/941186b433d08c6550a1831dd7afc3e7edca33c4386b97a6a3
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

2021-08-20 01:13:27.567789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 01:13:27.577938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 01:13:27.578804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-08-20 01:13:27.588428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
mkdir per_trained_models

mkdir: cannot create directory ‘per_trained_models’: File exists


In [ ]:
cd per_trained_models

/drive/My Drive/Project_Code/per_trained_models


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2021-08-20 01:13:55--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.203.128, 2607:f8b0:400e:c05::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90453990 (86M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v1_fp 100%[===================>]  86.26M  78.9MB/s    in 1.1s    

2021-08-20 01:13:56 (78.9 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [90453990/90453990]



In [ ]:
!tar -xvf ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
mkdir tfrecords

mkdir: cannot create directory ‘tfrecords’: File exists


In [ ]:
# Create train data:
!python xml_to_tfrecord.py

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
df_imagedetails.to_csv('Araneae_labels.csv', index=None)

In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
mkdir training_checkpoints

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-08-20 01:38:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.131.108, 35.153.244.194, 34.192.220.41, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.131.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  15.3MB/s    in 0.9s    

2021-08-20 01:38:41 (15.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = "/content/drive/MyDrive/CV1_workshop/training_checkpoints"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b90fa9b58077.ngrok.io


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/CV1_workshop/training_checkpoints

In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
!python train_models.py

2021-08-20 01:39:03.041000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 01:39:03.048742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 01:39:03.049436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.7/dist-packages/absl/flags/_validators.py:356: UserWarning: Flag --model_dir has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  'command line!' % flag_name)
/usr/local/lib/python3.7/dist-packages/absl/flags/_validators.py:356: UserWarning: Fl

In [ ]:
cd /drive/MyDrive/Project_Code

/drive/MyDrive/Project_Code


In [ ]:
mkdir trained_models 

In [ ]:
!python convert_to_savedmodel.py

In [ ]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/Project_Code/trained_models/test_model1'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/drive/MyDrive/Project_Code/labels_pbtxt/class_labels.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

Loading model...

OSError: ignored

In [ ]:

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/content/drive/MyDrive/CV1_workshop/images_annotations/hard_hat_workers24.png'

print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
#image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('\n Done', detections)